In [1]:
import density_to_structure
from density_to_structure import Atom, Molecule, molecule_to_tensor
from rdkit import Chem
from rdkit.Chem import AllChem
import pickle

RDKit WARNING: [17:29:40] Enabling RDKit 2019.09.3 jupyter extensions


In [2]:
with open('data/structs', 'rb') as f:
    ligands = pickle.load(f)

In [3]:
def rdkit_to_molecule(rdkit_molecule):
    rdkit_molecule = Chem.AddHs(rdkit_molecule)
    Chem.AllChem.EmbedMolecule(rdkit_molecule)
    Chem.AllChem.UFFOptimizeMolecule(rdkit_molecule)
    conformer = rdkit_molecule.GetConformers()[0]
    rdkit_positions = conformer.GetPositions()
    
    atoms = []
    for i, rd_atom in enumerate(rdkit_molecule.GetAtoms()):
        atoms.append(Atom(rdkit_positions[i], rd_atom.GetAtomicNum()))
    return Molecule(atoms)
        

In [4]:
my = rdkit_to_molecule(ligands[0])

In [5]:
def molecule_to_rdkit(molecule):
    rdkit_mol = Chem.rdchem.RWMol()
    for atom in molecule.atoms:
        rdkit_mol.AddAtom(Chem.rdchem.Atom(atom.atomic_number))
    Chem.AllChem.EmbedMolecule(rdkit_mol)
    for i in range(len(rdkit_mol.GetAtoms())):
        rdkit_mol.GetConformer(0).SetAtomPosition(i, molecule.atoms[i].coordinates)
    return rdkit_mol

In [6]:
its = molecule_to_rdkit(my)

In [7]:
def rdkit_to_tensor(molecule):
    from rdkit import Chem
    from moleculekit.smallmol.smallmol import SmallMol
    from moleculekit.tools.voxeldescriptors import getVoxelDescriptors

    molecule = Chem.AddHs(molecule)
    Chem.AllChem.EmbedMolecule(molecule)
    Chem.AllChem.MMFFOptimizeMolecule(molecule)
    molecule = SmallMol(molecule)
    vox, center, shape = getVoxelDescriptors(molecule, voxelsize=0.5, buffer=1, boxsize=[20, 20, 20], center=[0,0,0])
    vox = vox.reshape((40, 40, 40, 8))
    return vox
    

In [8]:
rdkit_vox = rdkit_to_tensor(ligands[0])

In [9]:
my_vox = molecule_to_tensor(my)

RuntimeError: Pre-condition Violation
	getNumImplicitHs() called without preceding call to calcImplicitValence()
	Violation occurred on line 181 in file Code/GraphMol/Atom.cpp
	Failed Expression: d_implicitValence > -1
	RDKIT: 2019.09.3
	BOOST: 1_70


In [10]:
my